# Methanol and pig iron production system

In [ ]:
import bw2data as bd

In [ ]:
bd.projects.set_current("optimex_remind")

## Getting ecoinvent inputs:

In [ ]:
electricity_mv = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="market group for electricity, medium voltage", location="RER")
electricity_lv = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="market group for electricity, low voltage", location="RER")
heat = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="market for heat, district or industrial", location="DEU") # Process not available for RER
water_tap = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="market for tap water", location="Europe without Switzerland")
water_deionized = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="deionized water production, via reverse osmosis, from brackish water", location="RER")

dac_system = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="direct air capture system, solvent-based, 1MtCO2", location="RER")
dac_system_eol = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="treatment of direct air capture system, solvent-based, 1MtCO2", location="RER")

pem_stack = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="electrolyzer production, 1MWe, PEM, Stack", location="RER")
pem_stack_eol = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="treatment of electrolyzer stack, 1MWe, PEM", location="RER")
pem_bop = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="electrolyzer production, 1MWe, PEM, Balance of Plant", location="RER")
pem_bop_eol = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="treatment of electrolyzer balance of plant, 1MWe, PEM", location="RER")

methanol_production_facility = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="methanol production facility, construction", location="RER")

blast_furnace_production = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="market for blast furnace", location="GLO")
coke = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="market for coke", location="RoW")
hard_coal = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="market group for hard coal", location="RER")
iron_ore_concentrate = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="market for iron ore concentrate", location="World")
iron_sinter = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="iron sinter production", location="RER")
iron_pellet = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="market for iron pellet", location="GLO")
natural_gas = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="market group for natural gas, high pressure", location="DEU")

methanol_factory_ng = bd.get_node(database="ei312_REMIND-EU_SSP2_NDC_2020", name="market for methanol factory", location="GLO")

In [ ]:
co2 = bd.get_node(database="ecoinvent-3.12-biosphere", name="Carbon dioxide, in air")
co2_emission = bd.get_node(database="ecoinvent-3.12-biosphere", name="Carbon dioxide, non-fossil", categories=("air",))

particulate_matter_sm = bd.get_node(database="ecoinvent-3.12-biosphere", name="Particulate Matter, < 2.5 um", categories=("air",))
particulate_matter_md = bd.get_node(database="ecoinvent-3.12-biosphere", name="Particulate Matter, > 2.5 um and < 10um", categories=("air",))
particulate_matter_lg = bd.get_node(database="ecoinvent-3.12-biosphere", name="Particulate Matter, > 10 um", categories=("air",))

## Foreground Setup

In [ ]:
if "foreground" in bd.databases:
    del bd.databases["foreground"] # to make sure we create the foreground from scratch
foreground = bd.Database("foreground")
foreground.register()

### Products:

In [ ]:
methanol = foreground.new_node(
    name="methanol",
    code="methanol",
    unit="kg",
    type=bd.labels.product_node_default,
)
methanol.save()

iron = foreground.new_node(
    name="pig iron",
    code="pig iron",
    unit="kg",
    type=bd.labels.product_node_default,
)
iron.save()

hydrogen = foreground.new_node(
    name="hydrogen",
    code="hydrogen",
    unit="kg",
    type=bd.labels.product_node_default,
)
hydrogen.save()

captured_co2 = foreground.new_node(
    name="captured CO2",
    code="captured CO2",
    unit="kg",
    type=bd.labels.product_node_default,
)
captured_co2.save()

### Processes:

In [ ]:
from optimex.utils import infer_operation_td_from_limits, infer_construction_td_from_limits, infer_eol_td_from_limits

#### DAC

In [ ]:
dac = foreground.new_node(
    name="direct air carbon capture",
    code="direct air carbon capture",
    location="RER",
    type=bd.labels.process_node_default,
    operation_time_limits=(0,15),
)
dac.save()

In [ ]:
# operation
dac.new_edge(
    input=captured_co2,
    amount=1.0,
    type=bd.labels.production_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(dac),
).save()

dac.new_edge(
    input=electricity_mv,
    amount=0.345,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(dac),
    vintage_improvements={2020: 1, 2030: 0.96, 2040: 0.94, 2050: 0.93},
).save()

dac.new_edge(
    input=heat,
    amount=6.28,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(dac),
    vintage_improvements={2020: 1, 2030: 0.95, 2040: 0.92, 2050: 0.90},
).save()

dac.new_edge(
    input=water_tap,
    amount=3.437,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(dac),
).save()

dac.new_edge(
    input=co2,
    amount=-1.0,
    type=bd.labels.biosphere_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(dac),
).save()

# construction
dac.new_edge(
    input=dac_system,
    amount=5e-11, # 5e-11
    type=bd.labels.consumption_edge_default,
    temporal_distribution=infer_construction_td_from_limits(dac),
).save()

# end-of-life
dac.new_edge(
    input=dac_system_eol,
    amount=-5e-11, # 5e-11
    type=bd.labels.consumption_edge_default,
    temporal_distribution=infer_eol_td_from_limits(dac),
).save()

#### PEM Electrolysis

In [ ]:
pem = foreground.new_node(
    name="PEM Electrolysis",
    code="PEM Electrolysis",
    location="RER",
    type=bd.labels.process_node_default,
    operation_time_limits=(0,8),
)
pem.save()

In [ ]:
# operation
pem.new_edge(
    input=hydrogen,
    amount=1.0,
    type=bd.labels.production_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(pem),
).save()

pem.new_edge(
    input=electricity_lv,
    amount=54,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(pem),
    vintage_improvements={2020: 1, 2030: 0.97, 2040: 0.95, 2050: 0.94},
).save()

pem.new_edge(
    input=water_deionized,
    amount=14,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(pem),
).save()

# construction
pem.new_edge(
    input=pem_stack,
    amount=1.34989e-6,
    type=bd.labels.consumption_edge_default,
    temporal_distribution=infer_construction_td_from_limits(pem),
).save()

pem.new_edge(
    input=pem_bop,
    amount=3.37373e-7,
    type=bd.labels.consumption_edge_default,
    temporal_distribution=infer_construction_td_from_limits(pem),
).save()

# end-of-life
pem.new_edge(
    input=pem_stack_eol,
    amount=-1.34989e-6,
    type=bd.labels.consumption_edge_default,
    temporal_distribution=infer_eol_td_from_limits(pem),
).save()

pem.new_edge(
    input=pem_bop_eol,
    amount=-3.37373e-7,
    type=bd.labels.consumption_edge_default,
    temporal_distribution=infer_eol_td_from_limits(pem),
).save()

#### Carbon dioxide hydrogenation to methanol

In [ ]:
co2_hydrogenation = foreground.new_node(
    name="Carbon dioxide hydrogenation to methanol",
    code="Carbon dioxide hydrogenation to methanol",
    location="RER",
    type=bd.labels.process_node_default,
    operation_time_limits=(0,15),
)
co2_hydrogenation.save()

In [ ]:
# operation
co2_hydrogenation.new_edge(
    input=methanol,
    amount=1.0,
    type=bd.labels.production_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(co2_hydrogenation),
).save()

co2_hydrogenation.new_edge(
    input=hydrogen,
    amount=0.138975,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(co2_hydrogenation),
).save()

co2_hydrogenation.new_edge(
    input=captured_co2,
    amount=1.690523,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(co2_hydrogenation),
).save()

co2_hydrogenation.new_edge(
    input=electricity_lv,
    amount=0.302895,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(co2_hydrogenation),
    vintage_improvements={2020: 1, 2030: 0.98, 2040: 0.97, 2050: 0.96},
).save()

co2_hydrogenation.new_edge(
    input=water_tap,
    amount=0.81959,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(co2_hydrogenation),
).save()

co2_hydrogenation.new_edge(
    input=co2_emission,
    amount=0.32,
    type=bd.labels.biosphere_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(co2_hydrogenation),
    vintage_improvements={2020: 1, 2030: 0.98, 2040: 0.97, 2050: 0.96},
).save()

# construction
co2_hydrogenation.new_edge(
    input=methanol_production_facility,
    amount=12.89,
    type=bd.labels.consumption_edge_default,
    temporal_distribution=infer_construction_td_from_limits(co2_hydrogenation),
).save()

#### Blast furnace w/ carbon capture

In [ ]:
blast_furnace_cc = foreground.new_node(
    name="Blast furnace with carbon capture",
    code="Blast furnace with carbon capture",
    location="RER",
    type=bd.labels.process_node_default,
    operation_time_limits=(0,25),
)
blast_furnace_cc.save()

In [ ]:
total_co2_emission_per_kg_iron = 0.849
captured_co2_per_kg_iron = 0.7054 # Happrecht et al., 2025, SI Section 1.3.2
total_pm_sm_emission_per_kg_iron = 2.8723e-5
total_pm_md_emission_per_kg_iron = 1.5957e-6
total_pm_lg_emission_per_kg_iron = 1.5957e-6
pm_emission_reduction = 0.5 # PM reduction through co-capture, Choi, 2013; Singh et al., 2011

# operation
blast_furnace_cc.new_edge(
    input=iron,
    amount=1.0,
    type=bd.labels.production_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace_cc),
).save()

blast_furnace_cc.new_edge(
    input=captured_co2,
    amount=captured_co2_per_kg_iron,
    type=bd.labels.production_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace_cc),
).save()

blast_furnace_cc.new_edge(
    input=co2_emission,
    amount=total_co2_emission_per_kg_iron - captured_co2_per_kg_iron,
    type=bd.labels.biosphere_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace_cc),
).save()

blast_furnace_cc.new_edge(
    input=natural_gas,
    amount=2.71/36, # Happrecht et al., 2025, SI Section 1.3.2 w/ 36 MJ/m3
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace_cc),
).save()

blast_furnace_cc.new_edge(
    input=coke,
    amount=9.724,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace_cc),
).save()

blast_furnace_cc.new_edge(
    input=hard_coal,
    amount=0.15,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace_cc),
).save()

blast_furnace_cc.new_edge(
    input=iron_ore_concentrate,
    amount=0.15,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace_cc),
).save()

blast_furnace_cc.new_edge(
    input=iron_pellet,
    amount=0.4,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace_cc),
).save()

blast_furnace_cc.new_edge(
    input=iron_sinter,
    amount=1.05,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace_cc),
).save()

blast_furnace_cc.new_edge(
    input=particulate_matter_sm,
    amount=(1 - pm_emission_reduction) * total_pm_sm_emission_per_kg_iron,
    type=bd.labels.biosphere_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace_cc),
).save()

blast_furnace_cc.new_edge(
    input=particulate_matter_md,
    amount=(1 - pm_emission_reduction) * total_pm_md_emission_per_kg_iron,
    type=bd.labels.biosphere_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace_cc),
).save()

blast_furnace_cc.new_edge(
    input=particulate_matter_lg,
    amount=(1 - pm_emission_reduction) * total_pm_lg_emission_per_kg_iron,
    type=bd.labels.biosphere_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace_cc),
).save()

# construction
blast_furnace_cc.new_edge(
    input=blast_furnace_production,
    amount=1.333e-11,
    type=bd.labels.consumption_edge_default,
    temporal_distribution=infer_construction_td_from_limits(blast_furnace_cc),
).save()

#### Blast furnace w/o carbon capture

In [ ]:
blast_furnace = foreground.new_node(
    name="Blast furnace",
    code="Blast furnace",
    location="RER",
    type=bd.labels.process_node_default,
    operation_time_limits=(0,25),
)
blast_furnace.save()

In [ ]:
# operation
blast_furnace.new_edge(
    input=iron,
    amount=1.0,
    type=bd.labels.production_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace),
).save()

blast_furnace.new_edge(
    input=co2_emission,
    amount=total_co2_emission_per_kg_iron,
    type=bd.labels.biosphere_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace),
).save()

blast_furnace.new_edge(
    input=natural_gas,
    amount=2.71/36, # Happrecht et al., 2025, SI Section 1.3.2 w/ 36 MJ/m3
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace),
).save()

blast_furnace.new_edge(
    input=coke,
    amount=9.724,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace),
).save()

blast_furnace.new_edge(
    input=hard_coal,
    amount=0.15,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace),
).save()

blast_furnace.new_edge(
    input=iron_ore_concentrate,
    amount=0.15,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace),
).save()

blast_furnace.new_edge(
    input=iron_pellet,
    amount=0.4,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace),
).save()

blast_furnace.new_edge(
    input=iron_sinter,
    amount=1.05,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace),
).save()

blast_furnace.new_edge(
    input=particulate_matter_sm,
    amount=total_pm_sm_emission_per_kg_iron,
    type=bd.labels.biosphere_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace),
).save()

blast_furnace.new_edge(
    input=particulate_matter_md,
    amount=total_pm_md_emission_per_kg_iron,
    type=bd.labels.biosphere_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace),
).save()

blast_furnace.new_edge(
    input=particulate_matter_lg,
    amount=total_pm_lg_emission_per_kg_iron,
    type=bd.labels.biosphere_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(blast_furnace),
).save()

# construction
blast_furnace.new_edge(
    input=blast_furnace_production,
    amount=1.333e-11,
    type=bd.labels.consumption_edge_default,
    temporal_distribution=infer_construction_td_from_limits(blast_furnace),
).save()

#### Direct reduction of iron

In [ ]:
direct_reduction = foreground.new_node(
    name="Direct reduction of iron",
    code="Direct reduction of iron",
    location="RER",
    type=bd.labels.process_node_default,
    operation_time_limits=(0,25),
)
direct_reduction.save()

In [ ]:
dri_h2_consumption = 0.06264
dri_iron_pellet_consumption = 1.359733

# operation
direct_reduction.new_edge(
    input=iron,
    amount=1.0,
    type=bd.labels.production_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(direct_reduction),
).save()

direct_reduction.new_edge(
    input=co2_emission,
    amount=0.03271,
    type=bd.labels.biosphere_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(direct_reduction),
).save()

direct_reduction.new_edge(
    input=hydrogen,
    amount=dri_h2_consumption,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(direct_reduction),
).save()

direct_reduction.new_edge(
    input=natural_gas,
    amount=0.0358938,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(direct_reduction),
).save()

direct_reduction.new_edge(
    input=iron_pellet,
    amount=dri_iron_pellet_consumption,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(direct_reduction),
).save()

direct_reduction.new_edge(
    input=electricity_mv,
    amount=0.0192446 + dri_h2_consumption * 4.024497 + dri_iron_pellet_consumption * 0.27267, # incl. h2 and iron pellet preheating 
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(direct_reduction),
).save()

# construction
direct_reduction.new_edge(
    input=blast_furnace_production,
    amount=1.333e-11,
    type=bd.labels.consumption_edge_default,
    temporal_distribution=infer_construction_td_from_limits(direct_reduction),
).save()

#### Natural gas reforming

In [ ]:
ng_reforming = foreground.new_node(
    name="Natural gas reforming",
    code="Natural gas reforming",
    location="RER",
    type=bd.labels.process_node_default,
    operation_time_limits=(0,25),
)
ng_reforming.save()

In [ ]:
# operation
ng_reforming.new_edge(
    input=methanol,
    amount=1.0,
    type=bd.labels.production_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(ng_reforming),
).save()

ng_reforming.new_edge(
    input=co2_emission,
    amount=0.33424,
    type=bd.labels.biosphere_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(ng_reforming),
).save()


ng_reforming.new_edge(
    input=natural_gas,
    amount=0.8895,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(ng_reforming),
).save()

ng_reforming.new_edge(
    input=water_deionized,
    amount=0.355,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(ng_reforming),
).save()

ng_reforming.new_edge(
    input=electricity_mv,
    amount=0.0886,
    type=bd.labels.consumption_edge_default,
    operation=True,
    temporal_distribution=infer_operation_td_from_limits(ng_reforming),
).save()

# construction
ng_reforming.new_edge(
    input=methanol_factory_ng,
    amount=3.716e-11,
    type=bd.labels.consumption_edge_default,
    temporal_distribution=infer_construction_td_from_limits(ng_reforming),
).save()

## Optimex setup

### Process LCA inputs

In [ ]:
from datetime import datetime

dbs = {
    2020: bd.Database("ei312_REMIND-EU_SSP2_NDC_2020"),
    2030: bd.Database("ei312_REMIND-EU_SSP2_NDC_2030"),
    2040: bd.Database("ei312_REMIND-EU_SSP2_NDC_2040"),
    2050: bd.Database("ei312_REMIND-EU_SSP2_NDC_2050"),
    2075: bd.Database("ei312_REMIND-EU_SSP2_NDC_2075"),
    2100: bd.Database("ei312_REMIND-EU_SSP2_NDC_2100"),
}

# Add representative_time metadata for each database
for year, db in dbs.items():
    db.metadata["representative_time"] = datetime(year, 1, 1).isoformat()

In [ ]:
# processes = [p for p in foreground]
# exchanges = [exc for p in processes for exc in p.technosphere()]

# for exc in exchanges:
#     node=exc.input
#     if node["database"] == bd.Database("ecoinvent-3.12-biosphere").name:
#         continue
#     if node["type"] != bd.labels.chimaera_node_default:
#         continue
#     for db in dbs.values():
#         node_in_other_db = db.get(
#             name=node["name"],
#             product=node.get("reference product", None),
#             location=node["location"],
#         )
#         if node["code"] != node_in_other_db["code"]:
#             node_in_other_db["code"] = node["code"]
#             node_in_other_db.save()

In [ ]:
from bw_temporalis import TemporalDistribution
import numpy as np

years = range(2025, 2051)
rng = np.random.default_rng(25)

# methanol demand
trend_meoh = np.linspace(1, 1, len(years))
# noise_meoh = rng.normal(0, 4.0, len(years))
noise_meoh = rng.normal(0, 0, len(years))
amount_meoh = trend_meoh + noise_meoh

td_methanol = TemporalDistribution(
    date=np.array(
        [datetime(year, 1, 1).isoformat() for year in years],
        dtype="datetime64[s]",
    ),
    amount=amount_meoh * 1e6, # Mt scale
)

# iron demand
trend_iron = np.linspace(1, 1, len(years))
# noise_iron = rng.normal(0, 8.0, len(years))
noise_iron = rng.normal(0, 0, len(years))
amount_iron = trend_iron + noise_iron

td_iron = TemporalDistribution(
    date=np.array(
        [datetime(year, 1, 1).isoformat() for year in years],
        dtype="datetime64[s]",
    ),
    amount=amount_iron * 1e6, # Mt scale
)

functional_demand = {methanol: td_methanol, iron: td_iron}

In [ ]:
method_climate_change = (
    "ecoinvent-3.12",
    "IPCC 2021 (incl. biogenic CO2) no LT",
    "climate change: total (incl. biogenic CO2) no LT",
    "global warming potential (GWP100) no LT",
)

method_land_use = (
    "ecoinvent-3.12",
    "EF v3.1 no LT",
    "land use no LT",
    "soil quality index no LT",
)

method_particulate_matter = (
    "ecoinvent-3.12",
    "EF v3.1 no LT",
    "particulate matter formation no LT",
    "impact on human health no LT",
)

method_water_use = (
    "ecoinvent-3.12",
    "EF v3.1 no LT",
    "water use no LT",
    "user deprivation potential (deprivation-weighted water consumption) no LT",
)

In [ ]:
from optimex import lca_processor

lca_config = lca_processor.LCAConfig(
    demand=functional_demand,
    temporal={
        "start_date": datetime(2020, 1, 1),
        "temporal_resolution": "year",
        "time_horizon": 100,
    },
    characterization_methods=[
        {
            "category_name": "climate_change",
            "brightway_method": method_climate_change,
            "metric": "CRF",  # CRF
        },
        {
            "category_name": "particulate_matter",
            "brightway_method": method_particulate_matter,
        },
        {
            "category_name": "land_use",
            "brightway_method": method_land_use,
        },
        {
            "category_name": "water_use",
            "brightway_method": method_water_use,
        }
    ],
)

In [ ]:
from optimex import converter

lca_data_processor = lca_processor.LCADataProcessor(lca_config)
manager = converter.ModelInputManager()
optimization_model_inputs = manager.parse_from_lca_processor(lca_data_processor) 

In [ ]:
manager.save("model_inputs_2050.json") # if you want to save the model inputs to a file

### Set up optimization and run

In [ ]:
from optimex import converter
manager = converter.ModelInputManager()

manager.load_inputs("model_inputs_2050.json") # if you want to load the model inputs from a file

### No evolution

In [ ]:
existing_capacities = {
    ("Blast furnace", 2005): 0.5e6,
    ("Blast furnace", 2015): 0.5e6,
    ("Natural gas reforming", 2005): 0.5e6,
    ("Natural gas reforming", 2015): 0.5e6,
}

no_background_evolution_mapping = {('ei312_REMIND-EU_SSP2_NDC_2020', year): 1.0 for year in range(2020, 2051)}

optimization_model_inputs_no_evolution = manager.override(
    existing_capacity=existing_capacities,
    mapping=no_background_evolution_mapping,
)

In [ ]:
from optimex import optimizer

model_no_evolution = optimizer.create_model(
    optimization_model_inputs_no_evolution,
    name = "no_evolution",
    objective_category = "climate_change",
)

In [ ]:
m_no_evolution, obj_no_evolution, results_no_evolution = optimizer.solve_model(model_no_evolution, solver_name="gurobi", tee=False)

In [ ]:
from optimex import postprocessing
pp_no_evolution = postprocessing.PostProcessor(m_no_evolution, plot_config={"figsize": (14, 6)})

In [ ]:
pp_no_evolution.get_dynamic_inventory()
pp_no_evolution.df_dynamic_inventory.to_excel("dynamic_inventory_no_evolution.xlsx")


In [ ]:
pp_no_evolution.get_characterized_dynamic_inventory(base_lcia_method=method_climate_change, df_inventory=pp_no_evolution.df_dynamic_inventory)
pp_no_evolution.df_characterized_inventory.to_excel("characterized_inventory_no_evolution.xlsx")

In [ ]:
pp_no_evolution.plot_capacity_balance_all(detailed=True)

In [ ]:
pp_no_evolution.get_impacts()
pp_no_evolution.plot_impacts()

In [ ]:
pp_no_evolution.df_production.to_excel("production_no_evolution.xlsx")
pp_no_evolution.df_demand.to_excel("demand_no_evolution.xlsx")
pp_no_evolution.get_production_capacity().to_excel("capacity_no_evolution.xlsx")
pp_no_evolution.df_impacts.to_excel("impacts_no_evolution.xlsx")

### Background evolution

In [ ]:
manager.load_inputs("model_inputs_2050.json")

existing_capacities = {
    ("Blast furnace", 2005): 0.5e6,
    ("Blast furnace", 2015): 0.5e6,
    ("Natural gas reforming", 2005): 0.5e6,
    ("Natural gas reforming", 2015): 0.5e6,
}

optimization_model_inputs_bg_evolution = manager.override(
    existing_capacity=existing_capacities,
)

In [ ]:
from optimex import optimizer

model_bg_evolution = optimizer.create_model(
    optimization_model_inputs_bg_evolution,
    name="fg_evolution",
    objective_category="climate_change",
)

In [ ]:
m_bg_evolution, obj_bg_evolution, results_bg_evolution = optimizer.solve_model(model_bg_evolution, solver_name="gurobi", tee=False)

In [ ]:
from optimex import postprocessing
pp_bg_evolution = postprocessing.PostProcessor(m_bg_evolution, plot_config={"figsize": (14, 6)})

In [ ]:
pp_bg_evolution.plot_capacity_balance_all(detailed=True)

In [ ]:
pp_bg_evolution.get_impacts()
pp_bg_evolution.plot_impacts()

### Foreground and background evolution

In [ ]:
manager.load_inputs("model_inputs_2050.json") # if you want to load the model inputs from a file

existing_capacities = {
    ("Blast furnace", 2005): 0.5e6,
    ("Blast furnace", 2015): 0.5e6,
    ("Natural gas reforming", 2005): 0.5e6,
    ("Natural gas reforming", 2015): 0.5e6,
}

# Note: Foreground evolution parameters are now defined directly on exchanges
# using the `vintage_improvements` attribute (see foreground setup cells above)

optimization_model_inputs_fg_bg_evolution = manager.override(
    existing_capacity=existing_capacities,
)

In [ ]:
from optimex import optimizer

model_fg_bg_evolution = optimizer.create_model(
    optimization_model_inputs_fg_bg_evolution,
    name = "fg_bg_evolution",
    objective_category = "climate_change",
)

In [ ]:
m_fg_bg_evolution, obj_fg_bg_evolution, results_fg_bg_evolution = optimizer.solve_model(model_fg_bg_evolution, solver_name="gurobi", tee=False)

In [ ]:
from optimex import postprocessing
pp_fg_bg_evolution = postprocessing.PostProcessor(m_fg_bg_evolution, plot_config={"figsize": (14, 6)})

In [ ]:
pp_fg_bg_evolution.get_dynamic_inventory()
pp_fg_bg_evolution.df_dynamic_inventory.to_excel("dynamic_inventory_fg_bg_evolution.xlsx")

pp_fg_bg_evolution.get_characterized_dynamic_inventory(base_lcia_method=method_climate_change, df_inventory=pp_fg_bg_evolution.df_dynamic_inventory)
pp_fg_bg_evolution.df_characterized_inventory.to_excel("characterized_inventory_fg_bg_evolution.xlsx")

In [ ]:
pp_fg_bg_evolution.plot_capacity_balance_all(detailed=True)

In [ ]:
pp_fg_bg_evolution.get_impacts()
pp_fg_bg_evolution.plot_impacts()

In [ ]:
pp_fg_bg_evolution.df_production.to_excel("production_fg_bg_evolution.xlsx")
pp_fg_bg_evolution.df_demand.to_excel("demand_fg_bg_evolution.xlsx")
pp_fg_bg_evolution.get_production_capacity().to_excel("capacity_fg_bg_evolution.xlsx")
pp_fg_bg_evolution.df_impacts.to_excel("impacts_fg_bg_evolution.xlsx")

### Water constraint

In [ ]:
manager.load_inputs("model_inputs_2050.json")

existing_capacities = {
    ("Blast furnace", 2005): 0.5e6,
    ("Blast furnace", 2015): 0.5e6,
    ("Natural gas reforming", 2005): 0.5e6,
    ("Natural gas reforming", 2015): 0.5e6,
}

# Note: Foreground evolution parameters are now defined directly on exchanges
# using the `vintage_improvements` attribute (see foreground setup cells above)

start_year = 2025
end_year = 2051  # range is exclusive, so this covers up to 2060
reduction_rate = 0

base_water_limit = 300_000

optimization_model_inputs_water_constraint = manager.override(
    existing_capacity=existing_capacities,
    category_impact_limits={
        ("water_use", year): base_water_limit * ((1 - reduction_rate) ** (year - start_year)) for year in range(start_year, end_year)
    },
)

In [ ]:
from optimex import optimizer

model_water_constraint = optimizer.create_model(
    optimization_model_inputs_water_constraint,
    name = "water_constraint",
    objective_category = "climate_change",
)

In [ ]:
m_water_constraint, obj_water_constraint, results_water_constraint = optimizer.solve_model(model_water_constraint, solver_name="gurobi", tee=False) # choose solver here, e.g. "gurobi", "cplex", "glpk", etc.

In [ ]:
from optimex import postprocessing
pp_water_constraint = postprocessing.PostProcessor(m_water_constraint, plot_config={"figsize": (14, 6)})

In [ ]:
pp_water_constraint.plot_capacity_balance_all(detailed=True)

In [ ]:
pp_water_constraint.get_impacts()
pp_water_constraint.plot_impacts()

Interesting: switch from ng reforming for methanol fully to hydrogenation, 

### Iridium constraint

In [ ]:
iridium = bd.get_node(database="ecoinvent-3.12-biosphere", name="Iridium")

In [ ]:
manager.load_inputs("model_inputs_2050.json")

existing_capacities = {
    ("Blast furnace", 2005): 0.5e6,
    ("Blast furnace", 2015): 0.5e6,
    ("Natural gas reforming", 2005): 0.5e6,
    ("Natural gas reforming", 2015): 0.5e6,
}

# Note: Foreground evolution parameters are now defined directly on exchanges
# using the `vintage_improvements` attribute (see foreground setup cells above)

start_year = 2025
end_year = 2051  # range is exclusive, so this covers up to 2060
reduction_rate = 0

base_water_limit = 300_000

optimization_model_inputs_iridium_constraint = manager.override(
    existing_capacity=existing_capacities,
    category_impact_limits={
        ("water_use", year): base_water_limit * ((1 - reduction_rate) ** (year - start_year)) for year in range(start_year, end_year)
    },
    cumulative_flow_limits_max={
        iridium["code"]: 0.125,
    },    
)

In [ ]:
from optimex import optimizer

model_iridium_constraint = optimizer.create_model(
    optimization_model_inputs_iridium_constraint,
    name = "iridium_constraint",
    objective_category = "climate_change",
)

In [ ]:
m_iridium_constraint, obj_iridium_constraint, results_iridium_constraint = optimizer.solve_model(model_iridium_constraint, solver_name="gurobi", tee=False) # choose solver here, e.g. "gurobi", "cplex", "glpk", etc.

In [ ]:
from optimex import postprocessing
pp_iridium_constraint = postprocessing.PostProcessor(m_iridium_constraint, plot_config={"figsize": (14, 6)})

In [ ]:
pp_iridium_constraint.get_dynamic_inventory()
pp_iridium_constraint.df_dynamic_inventory.to_excel("dynamic_inventory_iridium_constraint.xlsx")

pp_iridium_constraint.get_characterized_dynamic_inventory(base_lcia_method=method_climate_change, df_inventory=pp_iridium_constraint.df_dynamic_inventory)
pp_iridium_constraint.df_characterized_inventory.to_excel("characterized_inventory_iridium_constraint.xlsx")

In [ ]:
import pandas as pd

from datetime import datetime, timedelta
from functools import partial

def round_datetime(date: datetime, resolution: str) -> datetime:
    """
    Round a datetime object based on a given resolution

    Parameters
    ----------
    date : datetime
        datetime object to be rounded
    resolution: str
        Temporal resolution to round the datetime object to. Options are: 'year', 'month', 'day' and
        'hour'.

    Returns
    -------
    datetime
        rounded datetime object
    """
    if resolution == "year":
        mid_year = pd.Timestamp(f"{date.year}-07-01")
        return (
            pd.Timestamp(f"{date.year+1}-01-01")
            if date >= mid_year
            else pd.Timestamp(f"{date.year}-01-01")
        )

    if resolution == "month":
        start_of_month = pd.Timestamp(f"{date.year}-{date.month}-01")
        next_month = start_of_month + pd.DateOffset(months=1)
        mid_month = start_of_month + (next_month - start_of_month) / 2
        return next_month if date >= mid_month else start_of_month

    if resolution == "day":
        start_of_day = datetime(date.year, date.month, date.day)
        mid_day = start_of_day + timedelta(hours=12)
        return start_of_day + timedelta(days=1) if date >= mid_day else start_of_day

    if resolution == "hour":
        start_of_hour = datetime(date.year, date.month, date.day, date.hour)
        mid_hour = start_of_hour + timedelta(minutes=30)
        return start_of_hour + timedelta(hours=1) if date >= mid_hour else start_of_hour

    raise ValueError("Resolution must be one of 'year', 'month', 'day', or 'hour'.")

plot_df = df_characterized_inventory.groupby(["date","activity"], as_index=False).sum() #pp_iridium_constraint.
plot_df["date"] = plot_df["date"].apply(
    partial(round_datetime, resolution="year")
)

final_data = (
    plot_df.groupby(["date", "activity"], as_index=False)["amount"]
    .sum()
    .pivot(index="date", columns="activity", values="amount")
)

In [ ]:
pp_iridium_constraint.plot_capacity_balance_all(detailed=True)

In [ ]:
pp_iridium_constraint.get_impacts()
pp_iridium_constraint.plot_impacts()

In [ ]:
pp_iridium_constraint.df_production.to_excel("production_iridium_constraint.xlsx")
pp_iridium_constraint.df_demand.to_excel("demand_iridium_constraint.xlsx")
pp_iridium_constraint.get_production_capacity().to_excel("capacity_iridium_constraint.xlsx")
pp_iridium_constraint.df_impacts.to_excel("impacts_iridium_constraint.xlsx")

### Plotting the resulting radiative forcing 

In [ ]:
import pyomo.environ as pyo 
from dynamic_characterization import characterize

fg_scale = getattr(pp.m, "scales", {}).get("foreground", 1.0)
inventory = {
    (p, e, t): pyo.value(pp.m.scaled_inventory[p, e, t]) * fg_scale
    for p in pp.m.PROCESS
    for e in pp.m.ELEMENTARY_FLOW
    for t in pp.m.SYSTEM_TIME
}

import pandas as pd

dynamic_inventory_df = pd.DataFrame.from_records(
    [(p, e, t, v) for (p, e, t), v in inventory.items()],
    columns=["activity", "flow", "date", "amount"]
).astype({
    "activity": "str",
    "flow": "str",
    # "date": "datetime64[s]",
    "amount": "float64"
})

dynamic_inventory_df['date'] = pd.to_datetime(dynamic_inventory_df['date'].astype(int), format='%Y')

dynamic_inventory_df["flow"] = dynamic_inventory_df["flow"].apply(
    lambda x: bd.Database("ecoinvent-3.12-biosphere").get(code=x).id
)

In [ ]:
df_characterized = characterize(
        dynamic_inventory_df,
        metric="radiative_forcing", # could also be GWP
        base_lcia_method=method_climate_change,
        time_horizon=100,
        fixed_time_horizon=True,
)

df_characterized['date'] = pd.to_datetime(df_characterized['date'])
df_grouped = (
    df_characterized
    .assign(date_rounded=(df_characterized['date'] + pd.offsets.MonthBegin(6)).dt.to_period('Y').dt.to_timestamp())
    .groupby('date_rounded')['amount'].sum()
    .reset_index()
)

df_grouped.plot(x='date_rounded', y='amount')